In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# MLflow
> Log your metrics and models

## Libraries

In [ ]:
import copy
import subprocess
import time

import lightgbm as lgb
import mlflow
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression
from utilsforecast.data import generate_series
from utilsforecast.losses import rmse, smape
from utilsforecast.evaluation import evaluate
from utilsforecast.feature_engineering import fourier

import mlforecast.flavor
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExponentiallyWeightedMean
from mlforecast.utils import PredictionIntervals

## Data setup

In [ ]:
freq = 'h'
h = 10
series = generate_series(5, freq=freq)
valid = series.groupby('unique_id', observed=True).tail(h)
train = series.drop(valid.index)
train, X_df = fourier(train, freq=freq, season_length=24, k=2, h=h)

## Parameters

In [ ]:
params = {
    'init': {
        'models': {
            'lgb': lgb.LGBMRegressor(
                n_estimators=50, num_leaves=16, verbosity=-1
            ),
            'lr': LinearRegression(),
        },
        'freq': freq,
        'lags': [24],
        'lag_transforms': {
            1: [ExponentiallyWeightedMean(0.9)],
        },
        'num_threads': 2,
    },
    'fit': {
        'static_features': ['unique_id'],
        'prediction_intervals': PredictionIntervals(n_windows=2, h=h),
    }
}

## Logging

If you have a tracking server, you can run `mlflow.set_tracking_uri(your_server_uri)` to connect to it.

In [ ]:
mlflow.set_experiment("mlforecast")
with mlflow.start_run() as run:
    train_ds = mlflow.data.from_pandas(train)
    valid_ds = mlflow.data.from_pandas(valid)
    mlflow.log_input(train_ds, context="training")
    mlflow.log_input(valid_ds, context="validation")
    logged_params = copy.deepcopy(params) 
    logged_params['init']['models'] = {
        k: (v.__class__.__name__, v.get_params())
        for k, v in params['init']['models'].items()
    }
    mlflow.log_params(logged_params)
    mlf = MLForecast(**params['init'])
    mlf.fit(train, **params['fit'])
    preds = mlf.predict(h, X_df=X_df)
    eval_result = evaluate(
        valid.merge(preds, on=['unique_id', 'ds']),
        metrics=[rmse, smape],
        agg_fn='mean',
    )
    models = mlf.models_.keys()
    logged_metrics = {}
    for _, row in eval_result.iterrows():
        metric = row['metric']
        for model in models:
            logged_metrics[f'{metric}_{model}'] = row[model]
    mlflow.log_metrics(logged_metrics)
    mlforecast.flavor.log_model(model=mlf, artifact_path="model", registered_model_name=None)
    model_uri = mlflow.get_artifact_uri("model")
    run_id = run.info.run_id

/Users/deven367/miniforge3/envs/nixtla/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/18 16:52:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/18 16:53:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `

## Load model

In [ ]:
# Load model using the model URI from the registry
fallback_uri = f"runs:/{run_id}/model"
try:
    loaded_model = mlforecast.flavor.load_model(model_uri=model_uri)
    print("Model loaded successfully from registry!")
except Exception as e:
    print(f"Failed to load from registry URI: {e}")
    # Fallback: try loading from run artifacts
    print(f"Trying fallback URI: {fallback_uri}")
    loaded_model = mlforecast.flavor.load_model(model_uri=fallback_uri)
    print("Model loaded successfully from run artifacts!")

results = loaded_model.predict(h=h, X_df=X_df, ids=[3])
results.head(2)

## PyFunc

In [ ]:
fallback_uri = f"runs:/{run_id}/model"
try: 
    loaded_pyfunc = mlforecast.flavor.pyfunc.load_model(model_uri=model_uri)
except Exception as e:
    print(f"Failed to load from registry URI: {e}")
    loaded_pyfunc = mlforecast.flavor.pyfunc.load_model(model_uri=fallback_uri)
# single row dataframe
predict_conf = pd.DataFrame(
    [
        {
            "h": h,
            "ids": [0, 2],
            "X_df": X_df,
            "level": [80]
        }
    ]
)
pyfunc_result = loaded_pyfunc.predict(predict_conf)
pyfunc_result.head(2)

## Model serving

In [ ]:
host = 'localhost'
port = '5001'

fallback_uri = f"runs:/{run_id}/model"
cmd = f'mlflow models serve -m {fallback_uri} -h {host} -p {port} --env-manager local'
print(f"Serving model from runs: {fallback_uri}")

# initialize server
process = subprocess.Popen(cmd.split())
time.sleep(5)

# single row dataframe. must be JSON serializable
predict_conf = pd.DataFrame(
    [
        {
            "h": h,
            "ids": [3, 4],
            "X_df": X_df.astype({'ds': 'str'}).to_dict(orient='list'),
            "level": [95]
        }
    ]
)
payload = {'dataframe_split': predict_conf.to_dict(orient='split', index=False)}

try:
    resp = requests.post(f'http://{host}:{port}/invocations', json=payload)
    print(pd.DataFrame(resp.json()['predictions']).head(2))
except Exception as e:
    print(f"Error making prediction request: {e}")
    print(f"Response status: {resp.status_code if 'resp' in locals() else 'No response'}")
finally:
    process.terminate()
    process.wait(timeout=10)